<a href="https://colab.research.google.com/github/ynusinovich/streamflow-prediction/blob/main/ARE/HYSETS_DARTS_timeseries_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
data_root = '/content/drive/My Drive/Streamflow Data/'

Mounted at /content/drive


In [2]:
# !pip install pyyaml==5.4.1
!pip install xarray==0.18.2
!pip install torch==1.8.1  # there appears to be a version conflict with the current version of pytorch and cuDNN
!pip install darts==0.21.0
# !pip install -U matplotlib==3.1.3
# !pip install cudnn==8.3.2

import xarray as xr
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 807 kB 29.3 MB/s 
  Attempting uninstall: xarray
    Found existing installation: xarray 0.20.2
    Uninstalling xarray-0.20.2:
      Successfully uninstalled xarray-0.20.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires xarray>=0.20, but you have xarray 0.18.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 2.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that

In [3]:
#@title Load Watershed Properties and Data

In [4]:
# load static properties
lcrop = True

# load properties
if lcrop:
    wsp = pd.read_csv(data_root+'/HYSETS_watershed_properties_cleaned_prairies.txt')
else:
    wsp = pd.read_csv(data_root+'/HYSETS_watershed_properties_cleaned.txt')
# some cleaning
wsp.drop(columns='Unnamed: 0', inplace=True)
wsp['watershed'] = wsp['Watershed_ID']
wsp.set_index('watershed', drop=True, inplace=True)
wsp

,Watershed_ID,Source,Name,Official_ID,Centroid_Lat_deg_N,Centroid_Lon_deg_E,Drainage_Area_km2,Elevation_m,Slope_deg,Gravelius,...,Land_Use_Crops_frac,Land_Use_Snow_Ice_frac,Flag_Land_Use_Extraction,Permeability_logk_m2,Porosity_frac,Flag_Subsoil_Extraction,Hydrometric_station_latitude,Hydrometric_station_longitude,Flag_Natural_Boundaries,Flag_ALL
watershed,,,,,,,,,,,,,,,,,,,,,
865,865,HYDAT,WILLOW CREEK AT HIGHWAY NO. 811,05AB046,49.75476,-113.40730,2509.8584,1262.0,5.7682,1.6076,...,0.2563,0.0000,1,-15.262326,0.186644,1,49.75476,-113.40730,1,1
867,867,HYDAT,LITTLE BOW RIVER ABOVE TRAVERS RESERVOIR,05AC034,50.20417,-112.97780,3490.0697,1034.6,1.2383,1.8040,...,0.8288,0.0000,1,-15.200000,0.190000,1,50.20417,-112.97780,1,1
868,868,HYDAT,LITTLE BOW RIVER AT HIGHWAY NO. 533,05AC930,50.35417,-113.54310,791.6734,1026.4,0.6497,1.4250,...,0.9049,0.0000,1,-15.200000,0.190000,1,50.35417,-113.54310,1,1
869,869,HYDAT,WATERTON RIVER NEAR WATERTON PARK,05AD003,49.11366,-113.83940,612.6766,1838.1,22.5613,1.6875,...,0.0011,0.0057,1,-15.231325,0.106944,1,49.11366,-113.83940,1,1
877,877,HYDAT,BELLY RIVER NEAR MOON RIVER ROAD,05AD934,49.72870,-113.12470,3584.4565,1382.8,8.0552,1.7799,...,0.4126,0.0022,1,-15.359595,0.160534,1,49.72870,-113.12470,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,1230,HYDAT,NETLEY CREEK NEAR PETERSFIELD,05OJ008,50.32722,-97.04417,641.0558,256.3,0.2838,1.2551,...,0.5614,0.0000,1,-11.800000,0.060000,1,50.32722,-97.04417,1,1
1273,1273,HYDAT,ICELANDIC RIVER NEAR RIVERTON,05SC002,50.96472,-97.03747,1236.7307,252.9,0.2952,1.6384,...,0.5162,0.0000,1,-11.803577,0.060137,1,50.96472,-97.03747,1,1
1274,1274,HYDAT,FISHER RIVER NEAR DALLAS,05SD003,51.35617,-97.51222,1706.3842,258.5,0.4215,1.5097,...,0.2668,0.0000,1,-11.800000,0.060000,1,51.35617,-97.51222,1,1


In [5]:
# load NetCDF file lazily
if lcrop: filename = '/HYSETS_2020_QC_stations_mod4_crop.nc'
else: filename = '/HYSETS_2020_QC_stations_mod3.nc'
xds = xr.open_dataset(data_root+filename, chunks={'watershed':1}, decode_timedelta=False, decode_times=True)

# use watershed IDs to slice out watersheds of interest
xds = xds.sel(watershed=wsp['Watershed_ID'].values)
# load properties into memory
for var in xds.data_vars.values():
    if var.ndim == 1:
        var.load()
xds

<xarray.Dataset>
Dimensions:         (time: 25202, watershed: 112)
Coordinates:
  * time            (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2018-12-31
  * watershed       (watershed) int32 865 867 868 869 ... 1273 1274 1283 2365
Data variables: (12/19)
    watershedID     (watershed) float64 865.0 867.0 ... 1.283e+03 2.365e+03
    drainage_area   (watershed) float64 2.51e+03 3.49e+03 ... 4.608e+03 1.97e+03
    centroid_lat    (watershed) float64 49.75 50.2 50.35 ... 51.36 53.82 49.02
    centroid_lon    (watershed) float64 -113.4 -113.0 -113.5 ... -97.78 -109.8
    elevation       (watershed) float64 1.262e+03 1.035e+03 ... 250.6 1.05e+03
    slope           (watershed) float64 5.768 1.238 0.6497 ... 0.4621 1.671
    ...              ...
    discharge_diff  (watershed) float32 410.9 63.78 104.9 ... 119.0 157.5 40.5
    pr_diff         (watershed) float32 109.2 94.51 89.68 ... 131.8 118.1 78.63
    tasmax_diff     (watershed) float32 70.7 71.93 71.99 ... 71.91 73.89 73.98
    tasmin_diff     (watershed) float32 61.56 61.77 63.43 ... 73.04 70.92 67.12
    zeros_010_diff  (watershed) uint16 0 0 0 0 0 279 0 ... 71 315 94 28 0 0 156
    mask_1000       (watershed, time) uint16 dask.array<chunksize=(1, 25202), meta=np.ndarray>
Attributes: (12/13)
    title:          Hydrometeorological Sandbox of the École de technologie s...
    summary:        Hydrometeorological Sandbox of the École de technologie s...
    institution:    Hydrology, Climate and Climate Change Laboratory (HC3) at...
    institute_id:   HC3-ETS
    contact:        Richard Arsenault: richard.arsenault@etsmtl.ca
    date_created:   2020-05-26
    ...             ...
    featureType:    timeSeries
    cdm_data_type:  station
    license:        ODC-BY
    keywords:       HYSETS, GHCND, GSIM, hydrology, North America, streamflow...
    conventions:    CF-1.6, ACDD-1.3
    activity:       HYSETS

In [6]:
#@title Extract Periods of Consecutive Valid Data

In [7]:
data_mask = xds.data_mask
if 'mask_1000' in xds: 
  ND = 1000
  mask_N = xds.mask_1000
else: 
  ND = 300
  mask_N = xds.mask_300


# some reorganization of data
ts_varnames = ('discharge', )
cov_varnames = ('pr', 'tasmin', 'tasmax')
ts_xds = xr.Dataset({varname:xds[varname] for varname in ts_varnames+cov_varnames})
ts_xds

<xarray.Dataset>
Dimensions:    (time: 25202, watershed: 112)
Coordinates:
  * time       (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2018-12-31
  * watershed  (watershed) int32 865 867 868 869 877 ... 1273 1274 1283 2365
Data variables:
    discharge  (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    pr         (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    tasmin     (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    tasmax     (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>

In [8]:
# determine means and variances by watershed for static covariats and normalization
sds = xr.Dataset(data_vars=None, coords={'watershed':xds.watershed})
for varname in ts_varnames + cov_varnames:
    data_var = xds[varname]
    # compute means/std
    sds[varname+'_mean'] = data_var.mean(dim=('time',)).load()
    sds[varname+'_std']  = data_var.std(dim=('time',)).load()
    sds[varname+'_min']  = data_var.min(dim=('time',)).load()
    sds[varname+'_max']  = data_var.max(dim=('time',)).load()

# add to properties dataframe
stats_df = sds.to_dataframe()

## assemble static dataframes with means & std
# join with existing properties by index
wsp = wsp.join(stats_df, on=None, how='left')
# validate
assert (np.isnan(wsp['discharge_mean'].values)).any() == False
assert (wsp['discharge_mean'].values > 0).all() == True

In [9]:
## trim properties for training

# dropable columns
str_cols = ['Source', 'Name', 'Official_ID']
flag_cols = ['Flag_ALL', 'Flag_Shape_Extraction', 'Flag_Terrain_Extraction', 'Flag_Land_Use_Extraction', 'Flag_Subsoil_Extraction', 'Flag_Natural_Boundaries']
drop_cols = ['Watershed_ID', 'Centroid_Lon_deg_E', 'Hydrometric_station_latitude', 'Hydrometric_station_longitude'] + flag_cols + str_cols

# trim props for static covariates
print('\nDropping columns for traing data:\n', *drop_cols)
wsp_train = wsp.drop(columns=drop_cols, inplace=False)
wsp_train


Dropping columns for traing data:
 Watershed_ID Centroid_Lon_deg_E Hydrometric_station_latitude Hydrometric_station_longitude Flag_ALL Flag_Shape_Extraction Flag_Terrain_Extraction Flag_Land_Use_Extraction Flag_Subsoil_Extraction Flag_Natural_Boundaries Source Name Official_ID


,Centroid_Lat_deg_N,Drainage_Area_km2,Elevation_m,Slope_deg,Gravelius,Perimeter,Aspect_deg,Land_Use_Forest_frac,Land_Use_Grass_frac,Land_Use_Wetland_frac,...,pr_min,pr_max,tasmin_mean,tasmin_std,tasmin_min,tasmin_max,tasmax_mean,tasmax_std,tasmax_min,tasmax_max
watershed,,,,,,,,,,,,,,,,,,,,,
865,49.75476,2509.8584,1262.0,5.7682,1.6076,283.798,118.6022,0.2480,0.3854,0.0077,...,0.0,109.203796,-2.299139,10.016271,-43.372089,18.187035,10.625340,11.139233,-32.697273,38.002228
867,50.20417,3490.0697,1034.6,1.2383,1.8040,375.464,105.8134,0.0079,0.0923,0.0029,...,0.0,94.505379,-2.114604,10.326642,-43.293694,18.479317,11.219891,11.972828,-32.647053,39.279213
868,50.35417,791.6734,1026.4,0.6497,1.4250,141.224,163.4672,0.0009,0.0117,0.0003,...,0.0,89.681526,-2.588431,10.382421,-43.836945,19.597807,10.750421,12.201200,-32.867092,39.123215
869,49.11366,612.6766,1838.1,22.5613,1.6875,147.343,287.0749,0.5341,0.2608,0.0026,...,0.0,179.686600,-1.649298,9.022515,-40.595585,17.983831,9.862430,10.689588,-31.722679,36.081886
877,49.72870,3584.4565,1382.8,8.0552,1.7799,375.794,61.3698,0.2013,0.2598,0.0158,...,0.0,117.652039,-1.377792,9.549591,-40.872219,17.297737,10.827139,11.038117,-33.588688,36.676922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,50.32722,641.0558,256.3,0.2838,1.2551,111.934,112.7949,0.1996,0.1074,0.0525,...,0.0,109.500000,-3.432139,14.134577,-45.599998,22.007324,7.724002,14.932825,-34.500000,37.203873
1273,50.96472,1236.7307,252.9,0.2952,1.6384,202.881,78.8539,0.2345,0.1281,0.0277,...,0.0,105.437805,-4.080882,14.225872,-49.371803,23.310789,7.342315,15.035342,-35.151405,37.984589
1274,51.35617,1706.3842,258.5,0.4215,1.5097,219.568,49.3099,0.4043,0.1775,0.0465,...,0.0,131.809982,-4.674377,14.303836,-50.333035,22.702597,7.375019,15.017003,-34.013149,37.892857


In [10]:
# find consecutive blocks of at least 300 days
import numba

@numba.jit(numba.uint16[:](numba.float32[:]))
def consecutive_zeros(input):
    output = np.zeros(input.shape, dtype=np.uint16)
    c = 0
    for i in range(input.size):
        if input[i] == 0:
            c += 1
            output[i] = c
        else:
            c = 0  # reset to zero at every interruption
    return output

In [11]:
# %%time
# loop over stations
from darts import TimeSeries
from collections import OrderedDict

verbosity = 2

test_ratio = 0.2

ts_series_train = OrderedDict()
cov_series_train = OrderedDict()
ts_series_test = OrderedDict()
cov_series_test = OrderedDict()
ws_stats = pd.DataFrame(columns=['ID','train_n','train_yrs','test_n',
                                 'test_yrs','test_ratio','total_yrs'])

# for wid in [1151]: 
# for wid in [915]: 
for i,wid in enumerate(xds.watershed.values):

    ## identify indices pf valid periods
    ws_days = mask_N.sel(watershed=wid).values
    wsd_del = np.diff(np.where(ws_days > 0, np.int8(1), np.int8(0),),)  # potentially larger arrays
    # find start points of valid periods
    start_idxs = np.nonzero(wsd_del > 0)[0] + 1
    if ws_days[0] > 0: start_idxs =np.concatenate(([0],start_idxs))
    # find end points of valid periods
    end_idxs = np.nonzero(wsd_del < 0)[0] + 1
    if ws_days[-1] > 0: end_idxs = np.concatenate((end_idxs,[ws_days.size]))
    assert end_idxs.size == start_idxs.size
    # verify length of each period
    prd_len = end_idxs - start_idxs
    #print(wid, prd_len)
    if verbosity and prd_len.size > 0: 
        if verbosity > 0: print('ID:', wid, 'Periods:', prd_len)
        else: print("\nWatershed ID:", wid)
        #print(wid, start_idxs, end_idxs)
    assert ( prd_len >= ND ).all()
    # test even more stringently
    test_idxs = np.stack((start_idxs, end_idxs), axis=1)
    assert ( np.diff( test_idxs.flatten() ) > 0 ).all(), test_idxs

    ## Generate Timeseries objects
    # select by watershed
    ts_ws = ts_xds.sel(watershed=wid)
    stat_ws = wsp_train.loc[wid]  # indexed by watershed ID
    
    ts_test_tmp = []
    cov_test_tmp = []
    ts_train_tmp = []
    cov_train_tmp = []
    ws_yrs_test = 0
    ws_yrs_train = 0

    # loop over time periods and generate TimeSeries
    for n,si,ei in zip(range(1,len(start_idxs)+1),start_idxs, end_idxs):
        if verbosity > 1: print('Period:',si,ei)

        # slice time
        tsx = ts_ws.isel(time=slice(si,ei))
        # check for consecutive zeros
        ts_values = tsx[ts_varnames[0]].values
        zeros = consecutive_zeros(ts_values).astype(np.float32)
        max_zeros = zeros.max()
        if max_zeros > 1:
            n_zero = (ts_values == 0).sum()
            zero_ratio = 100*n_zero/len(ts_values)
            zeros[zeros==0] = np.nan
            print("Skipping Period due to too many zeros.")
            if verbosity > 1:
                print(f"Consecutive Zeros (min/mean/max): {np.nanmin(zeros)}/{np.nanmean(zeros):.0f}/{max_zeros}; Total Zeros: {zero_ratio:2.0f}%")
        else:
            dti = pd.DatetimeIndex(tsx.time)
            # extract arrays (and optionally normalize)
            ts_vars = dict()
            for varname,ts_var in tsx.data_vars.items():
                # ts_vars[varname] = ( ts_var.values - stat_ws[varname+'_mean'] ) / stat_ws[varname+'_std']
                ts_vars[varname] = ts_var.values
            ts_vars['dayofyear'] = -1.415 * np.cos(dti.day_of_year.values * 2*np.pi / 365.25, dtype=np.float32)  # sort of normalize

            # create TimeSeries
            ts_data = np.stack([ts_vars[varname] for varname in ts_varnames], axis=1)
            ts_series = TimeSeries.from_times_and_values(dti, ts_data, static_covariates=stat_ws)
            cov_data = np.stack([ts_vars[varname] for varname in cov_varnames+('dayofyear',)], axis=1)
            cov_series = TimeSeries.from_times_and_values(dti, cov_data,)
            # assign to train or test
            ts_len = len(dti)
            if ts_len < 10000:
                if n % 2 == 0:                
                    ts_test_tmp.append(ts_series)
                    cov_test_tmp.append(cov_series)
                    ws_yrs_test += ts_len
                else:
                    ts_train_tmp.append(ts_series)
                    cov_train_tmp.append(cov_series)
                    ws_yrs_train += ts_len
            else:
                split_idx = int( test_ratio * ts_len)
                ts_test_tmp.append(ts_series[:split_idx])
                cov_test_tmp.append(cov_series[:split_idx])
                ws_yrs_test += split_idx  # length of the first part
                ts_train_tmp.append(ts_series[split_idx:])
                cov_train_tmp.append(cov_series[split_idx:])
                ws_yrs_train += (ts_len - split_idx)  # length of the second part

    # append list of TimeSeries if non-emplty
    if len(ts_train_tmp) > 0:
        ts_series_test[wid] = ts_test_tmp
        cov_series_test[wid] = cov_test_tmp
        ts_series_train[wid] = ts_train_tmp
        cov_series_train[wid] = cov_train_tmp
        # gather some stats (append to end of DF)
        ws_stats.loc[len(ws_stats)] = {'ID':wid, 'train_n':len(ts_train_tmp), 'train_yrs':ws_yrs_train, 
                                       'test_n':len(ts_test_tmp), 'test_yrs':ws_yrs_test, 'total_yrs':(ws_yrs_train+ws_yrs_test), 
                                       'test_ratio':ws_yrs_test/(ws_yrs_train+ws_yrs_test)}
    elif len(ts_test_tmp) > 0:
        raise NotImplementedError()

ws_stats.set_index('ID', inplace=True)

ws_yrs_train_sum = ws_stats['train_yrs'].sum()
ws_yrs_test_sum = ws_stats['test_yrs'].sum()
test_ratio = ws_yrs_test_sum/(ws_yrs_train_sum+ws_yrs_test_sum)
print(f"\nGenerated {len(ts_series_train)} training and {len(ts_series_test)} test TimeSeries objects with Covariates,\n" +
      f"for a total of {ws_yrs_train_sum/365.25:.0f} and {ws_yrs_test_sum/365.25:.0f} watershed-years, respectively ({int(test_ratio*100):d}% test).")

ID: 869 Periods: [24106]
Period: 0 24106
ID: 911 Periods: [1157 3608]
Period: 5752 6909
Period: 20498 24106
ID: 915 Periods: [2952]
Period: 2800 5752
ID: 933 Periods: [17473]
Period: 6999 24472
ID: 992 Periods: [1430]
Period: 20120 21550
ID: 993 Periods: [11145  2571  2971]
Period: 7724 18869
Skipping Period due to too many zeros.
Consecutive Zeros (min/mean/max): 1.0/2/5.0; Total Zeros:  0%
Period: 18882 21453
Skipping Period due to too many zeros.
Consecutive Zeros (min/mean/max): 1.0/3/5.0; Total Zeros:  1%
Period: 21866 24837
Skipping Period due to too many zeros.
Consecutive Zeros (min/mean/max): 1.0/5/9.0; Total Zeros:  0%
ID: 1022 Periods: [16015]
Period: 7726 23741
ID: 1023 Periods: [1374]
Period: 22367 23741
ID: 1053 Periods: [2021 1065 2166 4086 1113]
Period: 10286 12307
Skipping Period due to too many zeros.
Consecutive Zeros (min/mean/max): 1.0/4/9.0; Total Zeros:  1%
Period: 13031 14096
Skipping Period due to too many zeros.
Consecutive Zeros (min/mean/max): 1.0/3/5.0; Tot

In [12]:
ws_stats

,train_n,train_yrs,test_n,test_yrs,test_ratio,total_yrs
ID,,,,,,
869,1,19285,1,4821,0.199992,24106
911,1,1157,1,3608,0.757188,4765
915,1,2952,0,0,0.000000,2952
933,1,13979,1,3494,0.199966,17473
992,1,1430,0,0,0.000000,1430
1022,1,12812,1,3203,0.200000,16015
1023,1,1374,0,0,0.000000,1374
1053,2,3279,1,4086,0.554786,7365
1089,1,8474,0,0,0.000000,8474


In [13]:
## scale inputs
from darts.dataprocessing.transformers import StaticCovariatesTransformer, Scaler
from sklearn.preprocessing import StandardScaler

# fit static trafo
static_scaler = StaticCovariatesTransformer()
static_scaler.fit([l[0] for l in ts_series_train.values()]) # fit using one sample per watershed

## fit data scalers
ts_train_scaled = OrderedDict()
ts_test_scaled = OrderedDict()
cov_train_scaled = OrderedDict()
cov_test_scaled = OrderedDict()
ts_series_scaler = OrderedDict()
cov_series_scaler = OrderedDict()

# loop over watersheds
for wid in ws_stats.index:

    # scale static covariates
    ts_train = ts_series_train[wid]
    static_covars = static_scaler.transform(ts_train[0]).static_covariates

    # scale timeseries
    ts_trafo = Scaler(StandardScaler())
    ts_trafo.fit(ts_train)
    ts_series_scaler[wid] = ts_trafo
    tmp_list = ts_trafo.transform(ts_train)
    ts_train_scaled[wid] = [ts.with_static_covariates(static_covars) for ts in tmp_list]
    tmp_list = ts_trafo.transform(ts_series_test[wid])
    ts_test_scaled[wid] = [ts.with_static_covariates(static_covars) for ts in tmp_list]

    # scale covariates
    cov_train = cov_series_train[wid]
    cov_trafo = Scaler(StandardScaler())
    cov_trafo.fit(cov_train)
    cov_series_scaler[wid] = cov_trafo
    cov_train_scaled[wid] = cov_trafo.transform(cov_train)
    cov_test_scaled[wid] = cov_trafo.transform(cov_series_test[wid])

ts_trafo

Scaler

In [14]:
import gzip, pickle

lscale = True

if lscale:
    pickle_dict = {'ts_train_scaled':   ts_train_scaled,
                  'ts_test_scaled':    ts_test_scaled,
                  'cov_train_scaled':  cov_train_scaled,
                  'cov_test_scaled':   cov_test_scaled,
                  'static_scaler':     static_scaler,
                  'ts_series_scaler':  ts_series_scaler,
                  'cov_series_scaler': cov_series_scaler,           
                  'watershed_stats':   ws_stats,
                  }

    pickle_name = "hysets_preprocessed_timseries_scaled{}_v3.gz".format('_crop' if lcrop else '')

else:
    pickle_dict = {'ts_series_train':   ts_series_train,
                   'cov_series_train':  cov_series_train,
                   'ts_series_test':    ts_series_test,
                   'cov_series_test':   cov_series_test,
                   'static_scaler':     static_scaler,
                   'ts_series_scaler':  ts_series_scaler,
                   'cov_series_scaler': cov_series_scaler,           
                   'watershed_stats':   ws_stats,
                   }

    pickle_name = "hysets_preprocessed_timseries_with_scaler{}.gz".format('_crop' if lcrop else '')

with gzip.open(data_root+pickle_name, 'wb') as gz:
    pickle.dump(pickle_dict, gz)

In [15]:
#@title Load Pickled Data

import gzip, pickle

lscale = True

pickle_name = "hysets_preprocessed_timseries{}{}_v3.gz".format('_scaled' if lscale else '',
                                                            '_crop' if lcrop else '')

with gzip.open(data_root+pickle_name, 'rb') as gz:
    pickle_dict = pickle.load(gz)

if lscale:
    # load scaled timeseries
    ts_series_train = pickle_dict['ts_train_scaled']
    cov_series_train = pickle_dict[ 'cov_train_scaled']
    ts_series_test = pickle_dict['ts_test_scaled']
    cov_series_test = pickle_dict['cov_test_scaled']
    # also load scalers and stats
    static_scaler = pickle_dict['static_scaler']
    ts_series_scaler = pickle_dict['ts_series_scaler']
    cov_series_scaler = pickle_dict['cov_series_scaler']
    ws_stats = pickle_dict['watershed_stats']

else:
    # load original timeseries
    ts_series_train = pickle_dict['ts_series_train']
    cov_series_train = pickle_dict[ 'cov_series_train']
    ts_series_test = pickle_dict['ts_series_test']
    cov_series_test = pickle_dict['cov_series_test']


In [27]:
#@title Select Watershed or Flatten Lists

from itertools import chain

WSID = None
# WSID = 869

if WSID is None:
    # use all watersheds
    ts_train = list(chain.from_iterable(ts_series_train.values()))
    ts_test = list(chain.from_iterable(ts_series_test.values()))
    cov_train = list(chain.from_iterable(cov_series_train.values()))
    cov_test = list(chain.from_iterable(cov_series_test.values()))

else:
    # use specified watershed (and remove static covariates)
    ts_train = [ts.with_static_covariates(None) for ts in ts_series_train[WSID]]
    ts_test = [ts.with_static_covariates(None) for ts in ts_series_test[WSID]]
    cov_train = cov_series_train[WSID]
    cov_test = cov_series_test[WSID]


In [17]:
#@title Train a LSTM Model with the Data


In [18]:
# ## load existing LSTM model
# from darts.models import RNNModel

# VERSION = 1
# N = 25
# acc_type = 'GPU'

# filepath = data_root+f"/HYSETS_LSTM_v{VERSION}.{N}.{acc_type}.pt"

# model = RNNModel.load(filepath)
# print("Epochs trained:", model.epochs_trained)

In [17]:
## load model from checkpoint
from darts.models import RNNModel

# VERSION = 4
# acc_type = 'GPU'

VERSION = 7
acc_type = 'CPU'
WSID = 869

N = 10

model_name = f"HYSETS_LSTM_WS{WSID}_v{VERSION}.{acc_type}"

model = RNNModel.load_from_checkpoint(model_name=model_name, work_dir=data_root,)

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [18]:
model.model_params

OrderedDict([('model', 'LSTM'),
             ('hidden_dim', 32),
             ('n_rnn_layers', 1),
             ('dropout', 0.1),
             ('training_length', 370),
             ('input_chunk_length', 270),
             ('n_epochs', 10),
             ('pl_trainer_kwargs', {'accelerator': 'cpu'}),
             ('model_name', 'HYSETS_LSTM_WS869_v7.CPU'),
             ('work_dir', '/content/drive/My Drive/Streamflow Data/'),
             ('save_checkpoints', True)])

In [19]:
## Create a new LSTM model

from darts.models import RNNModel

WSID = None
# VERSION = 7
# acc_type = 'GPU'

VERSION = 8
acc_type = 'CPU'
# WSID = 869

if WSID:
    model_name = f"HYSETS_LSTM_WS{WSID}_v{VERSION}.{acc_type}"
else:
    model_name = f"HYSETS_LSTM_v{VERSION}.{acc_type}"

pl_trainer_kwargs = dict(accelerator=acc_type.lower())
if acc_type == 'GPU':
    pl_trainer_kwargs["gpus"] = [0]

# simplified version of the Kratzert model
model = RNNModel(
    model="LSTM",    
    input_chunk_length=270,
    training_length=370, # 280
    hidden_dim=32, # 256
    n_rnn_layers=1,
    dropout=0.1, # 0.4
    n_epochs=1,
    batch_size=256,
    pl_trainer_kwargs=pl_trainer_kwargs,
    model_name=model_name,
    work_dir=data_root,
    force_reset=False,
    save_checkpoints=True,
)

# train model for one epoch
model.fit(ts_train, future_covariates=cov_train, val_series=ts_test, val_future_covariates=cov_test, verbose=True, num_loader_workers=8)

N = 0
filepath = data_root + '/' + model_name + f"/checkpoints/HYSETS_LSTM_v{VERSION}.{N}.{acc_type}.pt"
model.save(filepath)
print(f"\n   ***   Saved trained model {model_name} to Google Drive   ***   \n",f"'{filepath}'")
print("Epochs trained:", model.epochs_trained)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightnin

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.



   ***   Saved trained model HYSETS_LSTM_v8.CPU to Google Drive   ***   
 '/content/drive/My Drive/Streamflow Data//HYSETS_LSTM_v8.CPU/checkpoints/HYSETS_LSTM_v8.0.CPU.pt'
Epochs trained: 1


In [31]:
# train model for some epochs
for i in range(10):
    N += 1
    print(f"\n   ***   Training Iteration {N}   ***")
    model.fit(ts_train, future_covariates=cov_train, val_series=ts_test, val_future_covariates=cov_test, verbose=True, num_loader_workers=8)
    if N%1 == 0:
        filepath = data_root + '/' + model_name + f"/checkpoints/HYSETS_LSTM_v{VERSION}.{N}.{acc_type}.pt"
        model.save(filepath)
        print(f"\n   ***   Saved trained model version {VERSION}.{N} to Google Drive (trained with {acc_type})   ***   \n",f"'{filepath}'")


   ***   Training Iteration 11   ***


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /content/drive/My Drive/Streamflow Data/HYSETS_LSTM_v8.CPU/checkpoints exists and is not empty.
  r

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.



   ***   Saved trained model version 8.20 to Google Drive (trained with CPU)   ***   
 '/content/drive/My Drive/Streamflow Data//HYSETS_LSTM_v8.CPU/checkpoints/HYSETS_LSTM_v8.20.CPU.pt'


In [ ]:
%%time
#@title Test Model Prediction

# sid = 0  # long
# sid = 2  # longer
# sid = 3  # short
fcst_len = 100
input_len = model.model_params['input_chunk_length']

n_ws = len(ts_series_test)
pred_list = []
val_list = []

for sid in range(n_ws):

    ts_test = ts_series_test[sid]
    cov_test = cov_series_test[sid]

    # generate prediction series    
    series_len = len(ts_test)
    if input_len+fcst_len > series_len:
        print(f"   ---   Skipping Timeseries {sid+1} (of {n_ws}): too short for forecast")
    else:
        print(f"   ***   Validating Timeseries {sid+1} (of {n_ws}); length: {series_len} ...")

        init_idxs = np.arange(input_len, series_len-fcst_len+1, dtype=np.int16)

        ts_list = [ts_test[ii-input_len:ii] for ii in init_idxs]
        val_ts = [ts_test[ii:ii+fcst_len] for ii in init_idxs]
        val_list.append(val_ts)  # used in next cell, alongside pred

        cov_list = [cov_test[ii-input_len:ii+fcst_len] for ii in init_idxs]

        # run model prediction
        #print(" running model prediction ...", end='')
        pred = model.predict(n=fcst_len, series=ts_list, future_covariates=cov_list, verbose=None,)
        pred_list.append(pred)
        #print(" done.")



In [ ]:
%%time
## compute SMAPE

assert len(pred_list) == len(val_list)
n_pred = len(pred_list)
smape = np.zeros((n_pred, fcst_len))
nse = np.zeros((n_pred, fcst_len))
n_fcst_array = np.zeros((n_pred,1))

# loop over timeseries
for sid in range(n_pred):
# for sid in range(6,8):

    # ts_xdv = ts_series_test[sid].data_array()
    # print("Test Series Time Period:", ts_xdv.time[0].values,ts_xdv.time[-1].values)
    pred = pred_list[sid]
    val_ts = val_list[sid]
    
    n_fcst = len(pred)
    n_fcst_array[sid,0] = n_fcst
    print(f"Validating Wateshed {sid+1} (of {n_ws}); Number of Forecasts: {n_fcst}")

    smape_array = np.zeros((n_fcst, fcst_len))
    nse_up = np.zeros((n_fcst, fcst_len))
    nse_down = np.zeros((n_fcst, fcst_len))

    for i in range(n_fcst):
        val_xdv = val_ts[i].data_array()
        fcst_xdv = pred[i].data_array()
        if i in (0, n_fcst-1):
            assert val_xdv.time[0].values == fcst_xdv.time[0].values, (val_xdv.time[0].values,fcst_xdv.time[0].values)
            assert val_xdv.time[-1].values == fcst_xdv.time[-1].values, (val_xdv.time[-1].values,fcst_xdv.time[-1].values)
            # print("  Forecast #{i}:", val_xdv.time[0].values,val_xdv.time[-1].values)    
            # print("  Forecast #{i}:", fcst_xdv.time[0].values,fcst_xdv.time[-1].values)
        # renormalize
        ts_mean = fcst_xdv.attrs['static_covariates']['discharge_mean'][0]
        ts_std = fcst_xdv.attrs['static_covariates']['discharge_std'][0]
        val_data = val_xdv.values.squeeze()*ts_std + ts_mean
        fcst_data = fcst_xdv.values.squeeze()*ts_std + ts_mean
        fcst_data = fcst_data.clip(0,None)
        # fcst_data[:] = val_data[0]  # test for persistence forecast
        denominator = (np.abs(val_data) + np.abs(fcst_data))
        den_0 = ( denominator == 0 ).sum()
        if den_0 > 0:
            print(f"Zero-streamflow encountered {den_0}x")
            denominator[denominator == 0] = 1  # number doesn't matter, since enumerator is also zero
        smape_array[i,:] = 200 * np.abs(val_data - fcst_data) / denominator        
        nse_up[i,:] = np.square(val_data - fcst_data)
        nse_down[i,:] = np.square(val_data - ts_mean)  # already zero-centered
        
    smape[sid,:] = smape_array.mean(axis=0)
    nse[sid,:] = 1 - ( nse_up.sum(axis=0) / nse_down.sum(axis=0) )

n_fcst_sum = n_fcst_array.sum()
smape_avg = np.sum(smape * n_fcst_array, axis=0) / n_fcst_sum
smape_med = np.median(smape, axis=0)
nse_avg = np.sum(nse * n_fcst_array, axis=0) / n_fcst_sum
nse_med = np.median(nse, axis=0)

results = {" Median SMAPE: ": smape_med, "Average SMAPE: ": smape_avg, "   Median NSE: ": nse_med,"  Average NSE: ": nse_avg,}
for name,values in results.items():
    print('\n'+name, end='')
    for n in values: print(f' {n:4.0f}%,' if 'SMAPE' in name else f' {n:5.2f},', end='')

Validating Wateshed 1 (of 36); Number of Forecasts: 4452
Validating Wateshed 2 (of 36); Number of Forecasts: 3239
Validating Wateshed 3 (of 36); Number of Forecasts: 3125
Validating Wateshed 4 (of 36); Number of Forecasts: 1860
Validating Wateshed 5 (of 36); Number of Forecasts: 2202
Validating Wateshed 6 (of 36); Number of Forecasts: 2834
Validating Wateshed 7 (of 36); Number of Forecasts: 696
Zero-streamflow encountered 1x
Zero-streamflow encountered 2x
Zero-streamflow encountered 3x
Zero-streamflow encountered 4x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamflow encountered 5x
Zero-streamfl

In [ ]:
#@title Previous Forecast
for name,values in results.items():
    print('\n'+name, end='')
    for n in values: print(f' {n:4.0f}%,' if 'SMAPE' in name else f' {n:5.2f},', end='')


 Median SMAPE:    19%,   29%,   37%,   44%,   51%,   57%,   65%,   71%,   76%,   80%,   83%,   86%,   87%,   89%,   90%,   92%,   95%,   99%,   99%,   99%,  100%,  101%,  102%,  104%,  105%,  106%,  107%,  108%,  109%,  109%,  110%,  110%,  111%,  111%,  112%,  112%,  112%,  112%,  112%,  113%,  113%,  113%,  113%,  114%,  115%,  116%,  118%,  119%,  120%,  121%,  122%,  122%,  123%,  123%,  123%,  124%,  124%,  124%,  124%,  124%,  124%,  124%,  124%,  125%,  125%,  125%,  126%,  127%,  128%,  129%,  130%,  131%,  131%,  131%,  131%,  131%,  131%,  131%,  131%,  131%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  131%,  131%,  131%,  131%,  131%,  132%,  132%,
Average SMAPE:    24%,   35%,   43%,   50%,   56%,   61%,   65%,   69%,   73%,   76%,   78%,   81%,   83%,   85%,   87%,   89%,   90%,   92%,   93%,   95%,   96%,   97%,   98%,  100%,  101%,  102%,  102%,  103%,  104%,  105%,  106%,  106%,  107%,  108%,  108%,  109%,  110%,  110%,  

In [ ]:
#@title Skill of Persistence Forecast
for name,values in results.items():
    print('\n'+name, end='')
    for n in values[1:]: print(f' {n:4.0f}%,' if 'SMAPE' in name else f' {n:5.2f},', end='')


 Median SMAPE:     9%,   15%,   20%,   24%,   28%,   31%,   34%,   37%,   40%,   43%,   45%,   48%,   49%,   51%,   53%,   54%,   56%,   58%,   60%,   62%,   63%,   64%,   65%,   67%,   67%,   69%,   70%,   71%,   72%,   74%,   74%,   75%,   76%,   77%,   78%,   80%,   81%,   81%,   82%,   83%,   83%,   83%,   84%,   85%,   86%,   87%,   88%,   88%,   89%,   90%,   91%,   91%,   92%,   93%,   94%,   94%,   95%,   96%,   96%,   95%,   94%,   94%,   94%,   94%,  100%,  100%,  101%,  101%,  101%,  102%,  102%,  102%,  103%,  103%,  104%,  104%,  105%,  106%,  107%,  107%,  108%,  108%,  109%,  109%,  109%,  109%,  110%,  110%,  110%,  110%,  111%,  111%,  112%,  112%,  112%,  111%,  111%,  111%,  111%,
Average SMAPE:     9%,   15%,   20%,   25%,   28%,   32%,   35%,   37%,   40%,   42%,   45%,   47%,   49%,   51%,   52%,   54%,   56%,   57%,   59%,   60%,   62%,   63%,   65%,   66%,   67%,   68%,   70%,   71%,   72%,   73%,   74%,   75%,   76%,   77%,   78%,   79%,   80%,   81%,   82%,  